In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time, math, pickle
import numpy as np
import argparse, pprint

from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display, HTML, Math
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *

from scripts.VBOATS import VBOATS

vboat = VBOATS()
vboat.testMaxGndStep = 16

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

In [2]:
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_1")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_2")

# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/misc/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_6/depth")

#################################################################################################
#################################################################################################

# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/tmp/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/distance_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/angle_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/misc_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/test_keyParams")

found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/test_pool_1")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/test_pool_2")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/misc/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_6/depth")

#################################################################################################
#################################################################################################

# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/test_cases")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion/huh")

sFImgs = sorted(found_imgs)

In [3]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

imgPathSelector = widgets.Select(options=sFImgs,value=str(sFImgs[0]),description='File:', layout=Layout(width='100%'))

lblSz1 = widgets.Label(value="Deadzone Sizes (U,V):", layout=layCenter)
sz1Selector = widgets.IntSlider(value=10,min=0,max=20,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=3,min=0,max=20,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Strip[0] Deadzone Sizes (U,V):", layout=layCenter)
sz3Selector = widgets.IntSlider(value=4,min=0,max=20,step=1,description=' ', layout = layStretch)
sz4Selector = widgets.IntSlider(value=5,min=0,max=20,step=1,description=' ', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=5,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.25,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=45,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=True, description='Use Test Pipeline', layout = layStretch)
flag2 = widgets.Checkbox(value=False, description='Show Computational Timing', layout = layStretch)
flag3 = widgets.Checkbox(value=True, description='Use Filtered Overlay', layout = layStretch)
flag4 = widgets.Checkbox(value=False, description='Show Colorful Obstacle Overlay', layout = layStretch)

In [116]:
def thresholding(_img, dx=0,dy=3, sz1=5,sz2=10,tu0=0.15,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=15,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True,
    flag_timing=False,flag3=True,flag4=True):
    
#     cntColor = (255,255,255)
    cntColor = (0,0,255)
    
    vboat.dW = sz2;        vboat.dH = sz1
    vboat.dead_x = dx;     vboat.dead_y = dy
    vboat.dThs = [tdu,tdu2,win1thr,tdv]
    vboat.dHplus = win1

    vboat.kszs[0] = (k11,k12)
    vboat.kszs[1] = (k21,k22)
    vboat.kszs[2] = (k31,k32)
    vboat.testLowCntThresh = k21
    vboat.testHighCntThresh = k31
    vboat.testRestThreshRatio = tdu
    vboat.vmap_search_window_height = searchWin

    vboat.pipelineTest(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
    
    display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)
    display_obstacles = cv2.applyColorMap(display_obstacles,cv2.COLORMAP_JET)
    [cv2.rectangle(display_obstacles,ob[0],ob[1],(255,255,255),1) for ob in vboat.obstacles]
    dists, angs = vboat.extract_obstacle_information()
    
    viz = vboat.generate_visualization(dists, angs,use_rgb=False)
    
    if flag3:
        tmpU = np.copy(vboat.umap_processed)
        tmpU = cv2.normalize(tmpU,None,8191.0)
        tmpV = np.copy(vboat.vmap_processed)
        tmpV = cv2.normalize(tmpV,None,8191.0)
        
        dispU = cv2.applyColorMap(tmpU,cv2.COLORMAP_PARULA)
        dispV = cv2.applyColorMap(tmpV,cv2.COLORMAP_PARULA)
        [cv2.drawContours(dispU, [cnt], 0, cntColor, 1) for cnt in vboat.filtered_contours]
        [cv2.circle(dispV,(pxl[0],pxl[1]),2,(255,0,255),-1) for pxl in vboat.ground_pxls]
    else:
        tmpU = np.copy(vboat.umap_deadzoned)
        dispU = cv2.normalize(tmpU,None,8191.0)
        tmpV = np.copy(vboat.vmap_raw)
        dispV = cv2.normalize(tmpV,None,16383.0)
    
    if flag4: display_obstacles = np.copy(viz)
    
    overlay = make_uv_overlay(display_obstacles,dispU,dispV,color_uvmaps=not flag3)
    overlay = cv2.cvtColor(overlay,cv2.COLOR_BGR2RGB)
    pplot(overlay,'Overlay')

#     dispU1,dispU2,compU = vboat.umap_displays()
#     dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
#     pplot(dispU2,'Umap Display')

#     dispV1,dispV2,compV = vboat.vmap_displays()

    dispV2 = np.copy(vboat.vmap_raw)
    dispV2 = cv2.normalize(dispV2,None,16383.0)
    dispV2 = cv2.applyColorMap(dispV2,cv2.COLORMAP_PARULA)
    for wins in vboat.windows_obstacles:
            for win in wins:
                cv2.rectangle(dispV2,win[0],win[1],(255,255,0), 1)
                
    [cv2.circle(dispV2,(pxl[0],pxl[1]),2,(255,0,255),-1) for pxl in vboat.ground_pxls]
    try: dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
    except: pass
    pplot(dispV2,'Vmap Display')

    

In [117]:
%reload_ext autoreload
plt.close('all')

p = interactive(thresholding,
    _img = imgPathSelector, dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag3 = flag3, flag4 = flag4)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-5], p.children[-4], p.children[-3], p.children[-2]], layout = layStretch))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
display(tabs)

VGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShTZWxlY3QoZGVzY3JpcHRpb249dSdGaWxlOicsIGluZGV4PTI0LCBsYXlvdXQ9TGF5b3V0KHdpZHRoPXUnMTAwJSfigKY=


In [56]:
print(ground_detected)

True


In [133]:
vmap = np.copy(vboat.vmap_raw)
vcopy = np.copy(vmap)
cv2.rectangle(vmap,(vmap.shape[1]-10,0),(vmap.shape[1],vmap.shape[0]),(0,0,0), cv2.FILLED)

tmp1 = vmap[:, 0:40]
tmp2 = vmap[:, 40:vmap.shape[0]]

_,tmp1 = cv2.threshold(tmp1, 5,255,cv2.THRESH_TOZERO)
_,tmp2 = cv2.threshold(tmp2, 7,255,cv2.THRESH_TOZERO)
vmap = np.concatenate((tmp1,tmp2), axis=1)
tmpVmap = np.copy(vmap)

window_size=[18,10]
ground_detected, mask, maskInv,mPxls, ground_wins,_ = vboat.get_vmap_mask(vmap, maxStep = 19, min_ground_pixels = 8,
                                                threshold=15,window_size=window_size,shift_gain=0.65, verbose=True)

vmasked = cv2.bitwise_and(vmap,vmap,mask=maskInv)
vmap, newVmap = vmap_filter_tester(vmasked,verbose=True)

if not ground_detected:
    th,tw = newVmap.shape[:2]
    dead_strip = newVmap[:, 0:30]
    rest_strip = newVmap[:, 30:tw]
    tmpMax = np.max(rest_strip)
    tmpTH = int(tmpMax*0.7)

    tdh = (th*2)/3
    print(tmpMax2,tmpTH,tdh)

    topHalf = rest_strip[0:tdh, :]
    botHalf = rest_strip[tdh:th, :]

    _,topHalf = cv2.threshold(topHalf, tmpTH,255,cv2.THRESH_TOZERO)
    tmpRest = np.concatenate((topHalf,botHalf), axis=0)

    test = np.concatenate((dead_strip,tmpRest), axis=1)
else:
    test = np.copy(newVmap)
# _,test = cv2.threshold(newVmap, 7,255,cv2.THRESH_TOZERO)

pplots([vmasked, newVmap,test],"Vmap Compare", size=(1,3))

cp = np.copy(newVmap)
cp = cv2.normalize(cp,None,8191.0)
imgIn = cp
imgIn = np.copy(test)

disp = cv2.applyColorMap(imgIn,cv2.COLORMAP_PARULA)
disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
blank = np.copy(disp)

obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(imgIn, vboat.disparityBounds, vboat.xBounds,
                                                ground_detected=vboat.is_ground_present, verbose=True)
disps = []
for i,wins in enumerate(windows):
    tmp = np.copy(blank)
    for j, win in enumerate(wins):
        cv2.rectangle(tmp,win[0],win[1],(255,255,255), 1)
        cv2.rectangle(disp,win[0],win[1],(255,255,255), 1)
    disps.append(tmp)

# count = i + 1
print(len(windows))
# nrows = int(math.ceil(count/2.0))
# ncols = int(math.ceil(count/3.0))
# print(nrows,ncols)
# pplots(disps,"disps",(ncols,nrows+1))

disp_ground = cv2.applyColorMap(np.copy(vmap),cv2.COLORMAP_PARULA)
disp2 = cv2.applyColorMap(vcopy,cv2.COLORMAP_PARULA)
disp_ground = cv2.cvtColor(disp_ground,cv2.COLOR_BGR2RGB)
disp2 = cv2.cvtColor(disp2,cv2.COLOR_BGR2RGB)
[cv2.rectangle(disp_ground,win[0],win[1],(255,255,0), 1) for win in ground_wins]
[cv2.circle(disp_ground,(pxl[0],pxl[1]),2,(255,0,255),-1) for pxl in mPxls]

[cv2.rectangle(disp2,win[0],win[1],(255,255,0), 1) for win in ground_wins]
[cv2.circle(disp2,(pxl[0],pxl[1]),2,(255,0,255),-1) for pxl in mPxls]

pplot(disp,"disp")
pplots([disp_ground,disp2],"Ground Seg",(1,2))

[INFO] get_vmap_mask() ---- vmap shape (H,W,C): (480, 256)
[INFO] get_vmap_mask() ---- hist shape (H,W,C): (256,)
[INFO] get_vmap_mask() ---- Starting Location: 5, 460
[INFO] get_vmap_mask() ---- Starting Ground Search Technique...
	Window 0 [@ (4,451)]:	PxlCount = 45 - - dXmean = 1.000 - - newWinWidth = 5
	Window 1 [@ (4,442)]:	PxlCount = 36 - - dXmean = 0.000 - - newWinWidth = 5
	Window 2 [@ (4,433)]:	PxlCount = 36 - - dXmean = 0.000 - - newWinWidth = 5
	Window 3 [@ (4,424)]:	PxlCount = 36 - - dXmean = 0.000 - - newWinWidth = 5
	Window 4 [@ (5,415)]:	PxlCount = 45 - - dXmean = 1.000 - - newWinWidth = 5
	Window 5 [@ (5,406)]:	PxlCount = 52 - - dXmean = 0.000 - - newWinWidth = 5
	Window 6 [@ (5,397)]:	PxlCount = 43 - - dXmean = 0.000 - - newWinWidth = 5
	Window 7 [@ (5,388)]:	PxlCount = 43 - - dXmean = 0.000 - - newWinWidth = 5
	Window 8 [@ (5,379)]:	PxlCount = 51 - - dXmean = 0.000 - - newWinWidth = 5
	Window 9 [@ (6,370)]:	PxlCount = 50 - - dXmean = 1.000 - - newWinWidth = 5
	Window 

In [141]:
hist = np.sum(tmpVmap[0:350,:], axis=1)
plt.figure(14)
plt.plot(hist)
plt.show()
pplot(tmpVmap,"tmpVmap")

In [111]:
def vmap_filter_tester(vmap,verbose=True):
    
    vmasked = np.copy(vmap)
    vcopy = np.copy(vmasked)
#     pplot(vmasked,"vmasked")

    m = 5
    stripsV = strip_image(vmasked, nstrips=m,horizontal_strips=False)
    subStrips = strip_image(stripsV[0], nstrips=m,horizontal_strips=False)
#     pplots(stripsV,"stripsV", size=(1,m),flag_resize=True,scale=(1,5))
#     pplots(subStrips,"subStrips", size=(1,m),flag_resize=True,scale=(1,5))

    newStrips = []
    #======================================================================
    #                      FILTER STEP #1
    #======================================================================
    if verbose: print("----------- [Filter Step 1] -----------")
    strip = np.copy(stripsV[0]);    hs, ws = strip.shape[:2];       dw = ws / m
    # pplot(strip,"stripV0")

    dead_strip = strip[:, 0:dw];           oDead = np.copy(dead_strip)
    rest_strip = strip[:, dw:ws];          oRest = np.copy(rest_strip)

    hd, wd = dead_strip.shape[:2];          hr, wr = rest_strip.shape[:2]
    tmpdead = dead_strip[:, 0:3];        tmprest = dead_strip[:, 3:hd]

    vmapMax = np.max(vmasked); vstripMax = np.max(strip)
    if verbose: print("\tvmapMax, vstripMax: %d, %d" % (vmapMax,vstripMax))

    mx0 = np.max(dead_strip);       mx1 = np.max(tmpdead);     mx2 = np.max(tmprest)
    mxs = [mx0,mx1,mx2]
    if verbose: plist("\tMaxs:\t",mxs)
    mn0 = np.mean(dead_strip);      mn1 = np.mean(tmpdead);     mn2 = np.mean(tmprest)
    mns = [mn0,mn1,mn2]
    if verbose: plist("\tMeans:\t",mns)
    relMaxRatio = mx0/float(vstripMax)
    tmpRatio0 = np.nan_to_num(mn0/float(mx0));     tmpRatio1 = np.nan_to_num(mn1/float(mx1));     tmpRatio2 = np.nan_to_num(mn2/float(mx2))
    tmpRatio3 = tmpRatio1 + tmpRatio1*tmpRatio0
    tmpRatio4 = relMaxRatio + relMaxRatio*tmpRatio0
    rats = [relMaxRatio, tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3,tmpRatio4]
    if verbose: plist("\tRatios:\t",rats)

    tmpTH0 = int((1-tmpRatio0)*mx0)
    tmpTH1 = int(tmpRatio1*mx1)
    tmpTH2 = int(tmpRatio2*mx2)
    tmpTH3 = int(tmpRatio3*mx1)
    tmpTH4 = int((1-tmpRatio3)*mx1)
    tmpTH5 = int(tmpRatio4*mx2)
    ths = [tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5]
    if verbose: plist("\tThresholds:\t",ths)

    
    if relMaxRatio >= 1.0: 
        if not vboat.is_ground_present: tmpTH5 = int(0.15*vstripMax)
        else: tmpTH5 = int(0.3*vstripMax)
    
#     if verbose: print("\tDeadzone Thresholds: %d, %d" % (tmpTH4,tmpTH5))
    if verbose: print("\tDeadzone Thresholds: %d, %d" % (tmpTH5,tmpTH3))
    
#     _,testD = cv2.threshold(tmpdead, tmpTH4,255,cv2.THRESH_TOZERO)
    _,testD = cv2.threshold(tmpdead, tmpTH5,255,cv2.THRESH_TOZERO)
#     _,testR = cv2.threshold(tmprest, tmpTH5,255,cv2.THRESH_TOZERO)
    _,testR = cv2.threshold(tmprest, tmpTH3,255,cv2.THRESH_TOZERO)
    dead_strip = np.concatenate((testD,testR), axis=1)
    if verbose: print

    pplots([oDead,tmpdead,tmprest,testD,testR],"Filter Step 1: Debug", size=(1,5),flag_resize=True,scale=(1,5))
    #======================================================================
    #                      FILTER STEP #2
    #======================================================================
    if verbose: print("----------- [Filter Step 2] -----------")
    testclahe = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(hs/4,wr/4))
    claheRest = testclahe.apply(rest_strip)

    mx0 = np.max(rest_strip);       mx1 = np.max(claheRest);#     mx2 = np.max(tmprest)
    mxs = [mx0,mx1]#,mx2]

    mn0 = np.mean(rest_strip);      mn1 = np.mean(claheRest);#     mn2 = np.mean(tmprest)
    mns = [mn0,mn1]#,mn2]

    maxGain = (mx1)/float(mx0)
    meanGain = (mn1-mn0)/float(mn0)
    if verbose: print("\tMean Gain:\t%.3f" % meanGain)
    if verbose: print("\tMax Gain:\t%.3f" % maxGain)

    tmpRatio0 = (mn0*meanGain)/float(mx0);     tmpRatio1 = mn1/float(mx1)
    tmpRatio2 = tmpRatio0 + maxGain*tmpRatio0
    tmpRatio3 = tmpRatio1 + tmpRatio1*tmpRatio2

    rats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3]

    tmpTH0 = int((1-tmpRatio0)*mx1)
    tmpTH1 = int(tmpRatio1*mx1)
    tmpTH2 = int(tmpRatio2*mx1)
    tmpTH3 = int(tmpRatio0*mx1)
    ths = [tmpTH0,tmpTH1,tmpTH2,tmpTH3]

    if verbose: 
        plist("\tMaxs:\t",mxs)
        plist("\tMeans:\t",mns)
        plist("\tRatios:\t",rats)
        plist("\tThresholds:\t",ths)

    _,restMask = cv2.threshold(claheRest, tmpTH3,255,cv2.THRESH_TOZERO)
    tmpRest = cv2.bitwise_and(rest_strip,rest_strip,mask=restMask)
    newVstrip0 = np.concatenate((dead_strip,tmpRest), axis=1)

    pplots([dead_strip,rest_strip, claheRest,restMask,tmpRest],"Filter Step 2", size=(1,5),flag_resize=True,scale=(1,5))
#     pplots([strip, newVstrip0],"Filter Step 2: Compare", size=(1,2),flag_resize=True,scale=(1,5))
    newStrips.append(newVstrip0)
    if verbose: print

    #======================================================================
    #                      FILTER STEP #3
    #======================================================================
    if verbose: print("----------- [Filter Step 3] -----------")
    strip1 = np.copy(stripsV[1]);    hs1, ws1 = strip1.shape[:2];       dw1 = ws1 / m;  dh1 = hs1 / m
    # pplot(strip1,"stripV1")

    testclahe1 = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(dh1,dw1))
    claheRest1 = testclahe1.apply(strip1)

    mx0 = np.max(strip1);       mx1 = np.max(claheRest1);#     mx2 = np.max(tmprest)
    mxs = [mx0,mx1]#,mx2]

    mn0 = np.mean(strip1);      mn1 = np.mean(claheRest1);#     mn2 = np.mean(tmprest)
    mns = [mn0,mn1]#,mn2]

    maxGain = (mx1)/float(mx0)
    meanGain = (mn1-mn0)/float(mn0)
    if verbose: 
        print("\tMean Gain:\t%.3f" % meanGain)
        print("\tMax Gain:\t%.3f" % maxGain)

    relMaxRatio = mx0/float(vstripMax)
    tmpRatio0 = (mn0*meanGain)/float(mx0);     tmpRatio1 = mn1/float(mx1)
    tmpRatio2 = tmpRatio0 + relMaxRatio*tmpRatio0
    tmpRatio3 = tmpRatio1 + tmpRatio1*tmpRatio2

    rats = [relMaxRatio,tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3]

    tmpTH0 = int((1-tmpRatio0)*mx1)
    tmpTH1 = int(tmpRatio1*mx1)
    tmpTH2 = int(tmpRatio2*mx1)
    tmpTH3 = tmpTH2+tmpTH1
    tmpTH4 = int(tmpTH2 + tmpTH2*relMaxRatio)
    tmpTH5 = int((1-relMaxRatio)*mx0)
    ths = [tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5]
    if verbose: 
        plist("\tMaxs:\t",mxs)
        plist("\tMeans:\t",mns)
        plist("\tRatios:\t",rats)
        plist("\tThresholds:\t",ths)

    _,tmp = cv2.threshold(strip1, tmpTH5,255,cv2.THRESH_TOZERO)
#     _,tmpMask = cv2.threshold(claheRest1, tmpTH4+10,255,cv2.THRESH_TOZERO)
    _,tmpMask = cv2.threshold(claheRest1, tmpTH0,255,cv2.THRESH_TOZERO)
    tmpStrip = cv2.bitwise_and(strip1,strip1,mask=tmpMask)
    
    topHalf = tmpStrip[0:hs1/2, :]
    botHalf = tmpStrip[hs1/2:hs1, :]
    
    _,topHalf = cv2.threshold(topHalf, tmpTH5,255,cv2.THRESH_TOZERO)
    tmpStrip = np.concatenate((topHalf,botHalf), axis=0)
    
    newVstrip1 = np.copy(tmpStrip)
#     newVstrip1 = np.copy(tmp)

    # pplots([oDead,dead_strip, rest_strip,test,restMask,tmpRest],"tmpStrips", size=(1,6),flag_resize=True,scale=(1,5))
    pplots([strip1,claheRest1,tmpMask,tmpStrip,tmp],"Filter Step 3: Debug", size=(1,5),flag_resize=True,scale=(1,5))
    pplots([strip1, newVstrip1],"Filter Step 3: Compare", size=(1,2),flag_resize=True,scale=(1,5))
    newStrips.append(newVstrip1)
    if verbose: print
    #======================================================================
    #                      FILTER STEP #4
    #======================================================================
    tmpStrips = stripsV[2:]
    # pplots(tmpStrips,"leftover Strips", size=(1,len(tmpStrips)),flag_resize=True,scale=(1,5))

    for strip in tmpStrips:
        _,tmpStr = cv2.threshold(strip, 60,255,cv2.THRESH_TOZERO)
        newStrips.append(tmpStr)

    newVmap = np.concatenate(newStrips, axis=1)
    vizVmap = np.copy(newVmap)
    _,vizVmap = cv2.threshold(vizVmap, 0,255,cv2.THRESH_BINARY)
    return np.copy(vmap), np.copy(newVmap)

In [ ]:
umap_strip_filtering_tester(stripId=1,deadzone_filtering=False)

In [ ]:
verbose = True;     n = 6; deadzone_filtering = False
umap = np.copy(vboat.umap_deadzoned);   stripsU = strip_image(umap, nstrips=n);    strip = np.copy(stripsU[1])
# Stripping
hs, ws = strip.shape[:2];              dh = hs / n
print("strip.shape: ",strip.shape[:2])
print(dh)

if deadzone_filtering:
    dead_strip = strip[0:dh, :];           oDead = np.copy(dead_strip)
    rest_strip = strip[dh:hs, :];          oRest = np.copy(rest_strip)
    print("Shapes -- deadStrip, restStrip: %s, %s" % (str(dead_strip.shape),str(rest_strip.shape)))

    hd, wd = dead_strip.shape[:2];          hr, wr = rest_strip.shape[:2]
    umapMax = np.max(umap); stripMax = np.max(strip); preDeadMax = np.max(dead_strip); restMax = np.max(rest_strip)
    print("umapMax, stripMax,preDeadMax, restMax: %d, %d, %d, %d" % (umapMax, stripMax,preDeadMax,restMax))

    tmpdead = dead_strip[0:3, :];        tmprest = dead_strip[3:hd, :]
    tmaxs = [np.max(tmpdead), np.max(tmprest)]
    plist("Pre-Maxs:\t",tmaxs)

    ddzoneCopy=np.copy(tmpdead); restCopy = np.copy(tmprest)
    tmpdeadD = tmpdead[0:1, :];     tmprestD = tmpdead[1:tmpdead.shape[0], :]
    # pplots([tmpdead, tmprest],"Before Deadzone Strip Filtering", size=(2,1))
    pplots([tmpdead, tmprest],"Before Deadzone Strip Filtering", size=(2,1),flag_resize=True,scale=(5,1))
    # pplots([tmpdeadD, tmprestD],"Before Deadzone Deadzone Filtering", size=(2,1),flag_resize=True,scale=(5,1))

    mx0 = np.max(ddzoneCopy)
    mx1 = np.max(tmpdeadD)
    mx2 = np.max(tmprestD)

    mn0 = np.mean(ddzoneCopy)
    mn1 = np.mean(tmpdeadD)
    mn2 = np.mean(tmprestD)
    # print(mn0,mn1,mn2,mx0,mx1,mx2)

    tmpRatio0 = mn0/float(mx0)
    tmpRatio1 = mn1/float(mx1)
    tmpRatio2 = mn2/float(mx2)
    tmpRatio3 = tmpRatio2 + tmpRatio0*tmpRatio2
    # print(tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3)

    tmpTH0 = int((1-tmpRatio0)*mx0)
    tmpTH1 = int(tmpRatio1*mx1)
    tmpTH2 = int(tmpRatio2*mx2)
    tmpTH3 = int(tmpRatio0*mx0)
    tmpTH4 = int(tmpRatio3*mx0)
    plist("First Threshold Set:\t",[tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4])

    tt1 = ddzoneCopy[0:1, :];     tt2 = ddzoneCopy[1:2, :]
    tt3 = ddzoneCopy[2:ddzoneCopy.shape[0], :]

    _,testttt1 = cv2.threshold(tt1, tmpTH0,255,cv2.THRESH_TOZERO)
    _,testttt2 = cv2.threshold(tt2, tmpTH4+10,255,cv2.THRESH_TOZERO)
    _,testttt3 = cv2.threshold(tt3, tmpTH4,255,cv2.THRESH_TOZERO)

    testD = np.concatenate((testttt1,testttt2,testttt3), axis=0);
    tmpdead2 = np.copy(testD)

    mx0 = np.max(restCopy);     mn0 = np.mean(restCopy)
    mx1 = np.max(testD);        mn1 = np.mean(testD)
    gain = mx0/float(mx1)
    print("Max Gain:\t%.3f" % gain)

    tmpRatio0 = mn0/float(mx0)
    tmpRatio1 = mn1/float(mx1)
    tmpRatio2 = tmpRatio1+tmpRatio0
    tmpRatio3 = tmpRatio0 + gain

    tmpTH0 = int((1-tmpRatio0)*mx0)
    tmpTH1 = int(math.ceil(tmpRatio1*mx1))
    tmpTH3 = int(tmpRatio3*tmpTH0)
    tmpTH4 = int(tmpRatio3*mx0)
    tmpTH5 = tmpTH3 + tmpTH4
    plist("Second Threshold Set:\t",[tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5])
    combinedThresh = tmpTH5
    restThresh = tmpTH1

    if restThresh <= 2: restThresh = 1

    print(vboat.is_ground_present)
    if not vboat.is_ground_present or 100 < combinedThresh < 255: testThresh1 = tmpTH4
    elif combinedThresh >= 255: testThresh1 = int(0.5*255)
    else: testThresh1 = tmpTH5

    print("testThresh1: %d" % testThresh1)

    _,tmprest = cv2.threshold(tmprest, testThresh1,255,cv2.THRESH_TOZERO)
    # _,tmprest = cv2.threshold(tmprest, tmpTH5,255,cv2.THRESH_TOZERO)

    # pplots([tmpdead2,tmprest],"Thresholded Deadzone Strips w/ CLAHE",size=(2,1),flag_resize=True,scale=(5,1))
    # pplots([restCopy,tmprest],"Rest of deadzone",size=(2,1),flag_resize=True,scale=(5,1))

    ksz2 = (20,1);        ksz1 = (30,1)
    kernelr = cv2.getStructuringElement(cv2.MORPH_RECT,ksz1)
    kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,ksz2)

    tmpdead2 = cv2.morphologyEx(tmpdead2, cv2.MORPH_CLOSE, kernelr)
    tmprest = cv2.morphologyEx(tmprest, cv2.MORPH_CLOSE, kernelD)
    pplots([tmpdead2,tmprest],"Thresholded Deadzone Strips w/ CLAHE and Morphology",size=(2,1),flag_resize=True,scale=(5,1))

    dead_strip = np.concatenate((tmpdead2,tmprest), axis=0);
    oStrip = np.concatenate((dead_strip,rest_strip), axis=0);
    oDead2 = np.copy(dead_strip)

    # pplots([dead_strip],"Tmp",size=(1,1),flag_resize=True,scale=(5,1))
    pplot(dead_strip,"Tmp")
else:
    oStrip = np.copy(strip)
    h0 = 0
    restThresh = 1
    
tCLAH2 = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(hs,ws/4))
tstr = tCLAH2.apply(oStrip)

mx0 = np.max(tstr);     mn0 = np.mean(tstr)
mx1 = np.max(oStrip);   mn1 = np.mean(oStrip)
maxGain = (mx1)/float(mx0)
meanGain = (mn0 - mn1)/float(mn1)
print("Means:\t\t%.2f, %.2f"% (mn0,mn1))
print("Maxs:\t\t%d, %d"% (mx0,mx1))
print("Mean Gain:\t%.3f" % meanGain)
print("Max Gain:\t%.3f" % maxGain)

tmpRatio0 = mn0/float(mx0)
tmpRatio1 = mn1/float(mx1)
tmpRatio2 = tmpRatio1*meanGain
tmpRatio3 = tmpRatio0 + tmpRatio2
tmpRatio4 = tmpRatio0 + maxGain
tmpRatio5 = tmpRatio1+tmpRatio1*tmpRatio4
trats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3,tmpRatio4,tmpRatio5]
plist("Tmp Ratios:\t",trats,dplace=3)

tmpTH0 = int((1-tmpRatio0)*mx0)
tmpTH1 = int(tmpRatio0*mx0)
tmpTH2 = int(tmpRatio3*mx0)
tmpTH3 = int(tmpTH2 + tmpRatio3*tmpTH1 + math.ceil(tmpRatio1*mx0))
tmpTH4 = int(tmpRatio4*mx0)
tmpTH5 = int(math.ceil(tmpRatio5*mx1))

tThs = [tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5]
plist("Tmp Threhsolds: ",tThs)

# testThresh = 128
if not deadzone_filtering:
    testThresh = tmpTH4
#     restThresh = tmpTH5
    restThresh = 5
else: testThresh = tmpTH3
    
print("testThresh: %d" % testThresh)

_,tstrMask = cv2.threshold(tstr, testThresh,255,cv2.THRESH_BINARY)
_,ttstr = cv2.threshold(tstr, testThresh,255,cv2.THRESH_TOZERO)

if deadzone_filtering: ks = (40,3)
else: ks = (20,2)
    
kernelM = cv2.getStructuringElement(cv2.MORPH_RECT,ks)

ttstrM = cv2.morphologyEx(ttstr, cv2.MORPH_CLOSE, kernelM)
tstrMask = cv2.morphologyEx(tstrMask, cv2.MORPH_CLOSE, kernelM)

tmp = np.copy(oStrip);     stre = cv2.bitwise_and(tmp,tmp,mask=tstrMask)

tmpH = stre.shape[0];      h0 = dead_strip.shape[0]
if deadzone_filtering:
    tmpH = stre.shape[0];      
    tmpStrip = np.concatenate((dead_strip,stre[h0:tmpH,:]), axis=0)
else:
    tmpStrip = np.copy(stre)

print(strip.shape, tmpStrip.shape)
pplots([tstr,ttstr,ttstrM,stre,tmpStrip],"1st Umap Strip Deadzone Filtered",(5,1))

newStrip = np.copy(tmpStrip)

print("RestThresh: %d" % restThresh)
_,newStrip = cv2.threshold(newStrip, restThresh,255,cv2.THRESH_TOZERO)

pStrip = np.copy(newStrip)
# Find Contours
contours,_ = vboat.find_contours(pStrip, 45.0)
disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
print("Found %d Objects in 1st Strip" % len(contours))
# pplots([stripD,stripR,copyCld,copyClr],"CLAHE Sections",(4,1))
pplots([strip,newStrip,disp],"Test Compare",(3,1))

In [ ]:
ttt = 0.035 + 0.035*0.554
print ttt, 83* ttt

In [ ]:
def umap_strip_filtering_tester(stripId = 0, deadzone_filtering = True, flag_plot=True, verbose = True):
    n = 6
    umap = np.copy(vboat.umap_deadzoned);   stripsU = strip_image(umap, nstrips=n);    strip = np.copy(stripsU[stripId])
    # Stripping
    hs, ws = strip.shape[:2];              dh = hs / n
    # print("strip.shape: ",strip.shape[:2])
    umapMax = np.max(umap); stripMax = np.max(strip);
    
    if deadzone_filtering:
        dead_strip = strip[0:dh, :];           oDead = np.copy(dead_strip)
        rest_strip = strip[dh:hs, :];          oRest = np.copy(rest_strip)
        # print("Shapes -- deadStrip, restStrip: %s, %s" % (str(dead_strip.shape),str(rest_strip.shape)))

        hd, wd = dead_strip.shape[:2];          hr, wr = rest_strip.shape[:2]
        preDeadMax = np.max(dead_strip); restMax = np.max(rest_strip)
        print("umapMax, stripMax,preDeadMax, restMax: %d, %d, %d, %d" % (umapMax, stripMax,preDeadMax,restMax))

        tmpdead = dead_strip[0:3, :];        tmprest = dead_strip[3:hd, :]
        tmaxs = [np.max(tmpdead), np.max(tmprest)]
        plist("Pre-Maxs:\t",tmaxs)

        ddzoneCopy=np.copy(tmpdead); restCopy = np.copy(tmprest)
        tmpdeadD = tmpdead[0:1, :];     tmprestD = tmpdead[1:tmpdead.shape[0], :]
        # pplots([tmpdead, tmprest],"Before Deadzone Strip Filtering", size=(2,1))
        pplots([tmpdead, tmprest],"Before Deadzone Strip Filtering", size=(2,1),flag_resize=True,scale=(5,1))
        # pplots([tmpdeadD, tmprestD],"Before Deadzone Deadzone Filtering", size=(2,1),flag_resize=True,scale=(5,1))

        mx0 = np.max(ddzoneCopy)
        mx1 = np.max(tmpdeadD)
        mx2 = np.max(tmprestD)

        mn0 = np.mean(ddzoneCopy)
        mn1 = np.mean(tmpdeadD)
        mn2 = np.mean(tmprestD)
        # print(mn0,mn1,mn2,mx0,mx1,mx2)

        tmpRatio0 = mn0/float(mx0)
        tmpRatio1 = mn1/float(mx1)
        tmpRatio2 = mn2/float(mx2)
        tmpRatio3 = tmpRatio2 + tmpRatio0*tmpRatio2
        # print(tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3)

        tmpTH0 = int((1-tmpRatio0)*mx0)
        tmpTH1 = int(tmpRatio1*mx1)
        tmpTH2 = int(tmpRatio2*mx2)
        tmpTH3 = int(tmpRatio0*mx0)
        tmpTH4 = int(tmpRatio3*mx0)
        plist("First Threshold Set:\t",[tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4])

        tt1 = ddzoneCopy[0:1, :];     tt2 = ddzoneCopy[1:2, :]
        tt3 = ddzoneCopy[2:ddzoneCopy.shape[0], :]

        _,testttt1 = cv2.threshold(tt1, tmpTH0,255,cv2.THRESH_TOZERO)
        _,testttt2 = cv2.threshold(tt2, tmpTH4+10,255,cv2.THRESH_TOZERO)
        _,testttt3 = cv2.threshold(tt3, tmpTH4,255,cv2.THRESH_TOZERO)

        testD = np.concatenate((testttt1,testttt2,testttt3), axis=0);
        tmpdead2 = np.copy(testD)

        mx0 = np.max(restCopy);     mn0 = np.mean(restCopy)
        mx1 = np.max(testD);        mn1 = np.mean(testD)
        gain = mx0/float(mx1)
        print("Max Gain:\t%.3f" % gain)

        tmpRatio0 = mn0/float(mx0)
        tmpRatio1 = mn1/float(mx1)
        tmpRatio2 = tmpRatio1+tmpRatio0
        tmpRatio3 = tmpRatio0 + gain

        tmpTH0 = int((1-tmpRatio0)*mx0)
        tmpTH1 = int(math.ceil(tmpRatio1*mx1))
        tmpTH3 = int(tmpRatio3*tmpTH0)
        tmpTH4 = int(tmpRatio3*mx0)
        tmpTH5 = tmpTH3 + tmpTH4
        plist("Second Threshold Set:\t",[tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5])
        combinedThresh = tmpTH5
        restThresh = tmpTH1

        if restThresh <= 2: restThresh = 1

        print(vboat.is_ground_present)
        if not vboat.is_ground_present or 100 < combinedThresh < 255: testThresh1 = tmpTH4
        elif combinedThresh >= 255: testThresh1 = int(0.5*255)
        else: testThresh1 = tmpTH5

        print("testThresh1: %d" % testThresh1)

        _,tmprest = cv2.threshold(tmprest, testThresh1,255,cv2.THRESH_TOZERO)
        # _,tmprest = cv2.threshold(tmprest, tmpTH5,255,cv2.THRESH_TOZERO)

        # pplots([tmpdead2,tmprest],"Thresholded Deadzone Strips w/ CLAHE",size=(2,1),flag_resize=True,scale=(5,1))
        # pplots([restCopy,tmprest],"Rest of deadzone",size=(2,1),flag_resize=True,scale=(5,1))

        ksz2 = (20,1);        ksz1 = (30,1)
        kernelr = cv2.getStructuringElement(cv2.MORPH_RECT,ksz1)
        kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,ksz2)

        tmpdead2 = cv2.morphologyEx(tmpdead2, cv2.MORPH_CLOSE, kernelr)
        tmprest = cv2.morphologyEx(tmprest, cv2.MORPH_CLOSE, kernelD)
        pplots([tmpdead2,tmprest],"Thresholded Deadzone Strips w/ CLAHE and Morphology",size=(2,1),flag_resize=True,scale=(5,1))

        dead_strip = np.concatenate((tmpdead2,tmprest), axis=0);
        oStrip = np.concatenate((dead_strip,rest_strip), axis=0);
        oDead2 = np.copy(dead_strip)

        # pplots([dead_strip],"Tmp",size=(1,1),flag_resize=True,scale=(5,1))
        pplot(dead_strip,"Tmp")
        h0 = dead_strip.shape[0]
    else:
        oStrip = np.copy(strip)
        h0 = 0
        restThresh = 1
        
    tCLAH2 = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(hs,ws/4))
    tstr = tCLAH2.apply(oStrip)

    mx0 = np.max(tstr);     mn0 = np.mean(tstr)
    mx1 = np.max(oStrip);   mn1 = np.mean(oStrip)
    meanGain = (mn0 - mn1)/float(mn1)
    # print("Means:\t\t%.2f, %.2f"% (mn0,mn1))
    # print("Maxs:\t\t%d, %d"% (mx0,mx1))
    # print("Mean Gain:\t%.3f" % meanGain)

    tmpRatio0 = mn0/float(mx0)
    tmpRatio1 = mn1/float(mx1)
    tmpRatio2 = tmpRatio1*meanGain
    tmpRatio3 = tmpRatio0 + tmpRatio2
    trats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3]
    plist("Tmp Ratios:\t",trats,dplace=3)

    tmpTH0 = int((1-tmpRatio0)*mx0)
    tmpTH1 = int(tmpRatio0*mx0)
    tmpTH2 = int(tmpRatio3*mx0)
    tmpTH3 = int(tmpTH2 + tmpRatio3*tmpTH1 + math.ceil(tmpRatio1*mx0))
    tThs = [tmpTH0,tmpTH1,tmpTH2,tmpTH3]
    plist("Tmp Threhsolds: ",tThs)

    # testThresh = 93
    testThresh = tmpTH3
    print("testThresh: %d" % testThresh)

    _,tstrMask = cv2.threshold(tstr, testThresh,255,cv2.THRESH_BINARY)
    _,ttstr = cv2.threshold(tstr, testThresh,255,cv2.THRESH_TOZERO)


    kernelM = cv2.getStructuringElement(cv2.MORPH_RECT,(40,3))
    tstrMask = cv2.morphologyEx(tstrMask, cv2.MORPH_CLOSE, kernelM)

    tmp = np.copy(oStrip);     stre = cv2.bitwise_and(tmp,tmp,mask=tstrMask)

    if deadzone_filtering:
        tmpH = stre.shape[0];      
        tmpStrip = np.concatenate((dead_strip,stre[h0:tmpH,:]), axis=0)
    else:
        tmpStrip = np.copy(stre)
        
    pplots([tstr,ttstr,tstrMask,stre,tmpStrip],"1st Umap Strip Deadzone Filtered",(5,1))

    newStrip = np.copy(tmpStrip)

    print("RestThresh: %d" % restThresh)
    _,newStrip = cv2.threshold(newStrip, restThresh,255,cv2.THRESH_TOZERO)

    pStrip = np.copy(newStrip)
    # Find Contours
    contours,_ = vboat.find_contours(pStrip, 45.0)
    disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
    [cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
    print("Found %d Objects in 1st Strip" % len(contours))
    # pplots([stripD,stripR,copyCld,copyClr],"CLAHE Sections",(4,1))
    pplots([strip,newStrip,disp],"Test Compare",(3,1))

    return pStrip

In [ ]:
def second_umap_strip_filter():
    verbose = True;     n = 6; deadzone_filtering = False
    umap = np.copy(vboat.umap_deadzoned);   stripsU = strip_image(umap, nstrips=n);    strip = np.copy(stripsU[1])
    # Stripping
    hs, ws = strip.shape[:2];              dh = hs / n

    tCLAH2 = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(hs,ws/4))
    tstr = tCLAH2.apply(oStrip)

    mx0 = np.max(tstr);     mn0 = np.mean(tstr)
    mx1 = np.max(oStrip);   mn1 = np.mean(oStrip)
    maxGain = (mx1)/float(mx0)
    meanGain = (mn0 - mn1)/float(mn1)
    print("Means:\t\t%.2f, %.2f"% (mn0,mn1))
    print("Maxs:\t\t%d, %d"% (mx0,mx1))
    print("Mean Gain:\t%.3f" % meanGain)
    print("Max Gain:\t%.3f" % maxGain)

    tmpRatio0 = mn0/float(mx0)
    tmpRatio1 = mn1/float(mx1)
    tmpRatio2 = tmpRatio1*meanGain
    tmpRatio3 = tmpRatio0 + tmpRatio2
    tmpRatio4 = tmpRatio0 + maxGain
    trats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3,tmpRatio4]
    plist("Tmp Ratios:\t",trats,dplace=3)

    tmpTH0 = int((1-tmpRatio0)*mx0)
    tmpTH1 = int(tmpRatio0*mx0)
    tmpTH2 = int(tmpRatio3*mx0)
    tmpTH3 = int(tmpTH2 + tmpRatio3*tmpTH1 + math.ceil(tmpRatio1*mx0))
    tmpTH4 = int(tmpRatio4*mx0)
    tmpTH5 = int((tmpRatio1+tmpRatio0)*mx1)

    tThs = [tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5]
    plist("Tmp Threhsolds: ",tThs)

    # testThresh = 128
    if not deadzone_filtering:
        testThresh = tmpTH4
        restThresh = tmpTH5
    else: testThresh = tmpTH3

    print("testThresh: %d" % testThresh)

    _,tstrMask = cv2.threshold(tstr, testThresh,255,cv2.THRESH_BINARY)
    _,ttstr = cv2.threshold(tstr, testThresh,255,cv2.THRESH_TOZERO)

    if deadzone_filtering: ks = (40,3)
    else: ks = (20,2)

    kernelM = cv2.getStructuringElement(cv2.MORPH_RECT,ks)

    ttstrM = cv2.morphologyEx(ttstr, cv2.MORPH_CLOSE, kernelM)
    tstrMask = cv2.morphologyEx(tstrMask, cv2.MORPH_CLOSE, kernelM)

    tmp = np.copy(oStrip);     stre = cv2.bitwise_and(tmp,tmp,mask=tstrMask)

    tmpH = stre.shape[0];      h0 = dead_strip.shape[0]
    if deadzone_filtering:
        tmpH = stre.shape[0];      
        tmpStrip = np.concatenate((dead_strip,stre[h0:tmpH,:]), axis=0)
    else:
        tmpStrip = np.copy(stre)

    print(strip.shape, tmpStrip.shape)
    pplots([tstr,ttstr,ttstrM,stre,tmpStrip],"1st Umap Strip Deadzone Filtered",(5,1))

    newStrip = np.copy(tmpStrip)

    print("RestThresh: %d" % restThresh)
    _,newStrip = cv2.threshold(newStrip, restThresh,255,cv2.THRESH_TOZERO)

    pStrip = np.copy(newStrip)
    # Find Contours
    contours,_ = vboat.find_contours(pStrip, 45.0)
    disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
    [cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
    print("Found %d Objects in 1st Strip" % len(contours))
    # pplots([stripD,stripR,copyCld,copyClr],"CLAHE Sections",(4,1))
    pplots([strip,newStrip,disp],"Test Compare",(3,1))

In [ ]:
mx0 = np.max(restCopy);     mn0 = np.mean(restCopy)
mx1 = np.max(testD);        mn1 = np.mean(testD)

mmxs = [mx0,mx1]
mmns = [mn0,mn1]
meanGain = (mn1 - mn0)/float(mn0)
gain = mx0/float(mx1)
plist("Means:\t\t",mmns)
plist("Maxs:\t\t", mmxs)
print("Mean Gain:\t%.3f" % meanGain)
print("Max Gain:\t%.3f" % gain)

tmpRatio0 = mn0/float(mx0)
tmpRatio1 = mn1/float(mx1)
# tmpRatio2 = tmpRatio1*meanGain
# tmpRatio3 = tmpRatio0 + tmpRatio2
tmpRatio2 = tmpRatio1+tmpRatio0
tmpRatio3 = tmpRatio0 + gain
trats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3]
# trats = [tmpRatio0,tmpRatio1,tmpRatio3]

plist("Tmp Ratios:\t",trats,dplace=4)

tmpTH0 = int((1-tmpRatio0)*mx0)
tmpTH1 = int(math.ceil(tmpRatio1*mx1))
tmpTH2 = int(tmpRatio2*mx0)
tmpTH3 = int(tmpRatio3*tmpTH0)
tmpTH4 = int(tmpRatio3*mx0)
tmpTH5 = tmpTH3 + tmpTH4
# tmpTH3 = int(tmpTH2 + tmpRatio3*tmpTH1 + math.ceil(tmpRatio1*mx0))
tThs = [tmpTH0,tmpTH1,tmpTH2,tmpTH3, tmpTH4,tmpTH5]
plist("Tmp Threhsolds: ",tThs)

# tt1 = ddzoneCopy[0:1, :];     tt2 = ddzoneCopy[1:ddzoneCopy.shape[0], :]

# _,testttt1 = cv2.threshold(tt1, tmpTH0,255,cv2.THRESH_TOZERO)
# _,testttt2 = cv2.threshold(tt2, tmpTH3,255,cv2.THRESH_TOZERO)

# testD = np.concatenate((testttt1,testttt2), axis=0);

# pplots([ddzoneCopy,testD],"Test ddzones", (2,1),flag_resize=True,scale=(5,1))

In [ ]:
37*(1-0.286)
0.65*255

In [ ]:
dm = vboat.dmax
pd = vboat.dmax/256.0
pd = 100/256.0
tmpN = pd*6
print(pd)
print(tmpN)
tmp = math.ceil(tmpN)
print(tmp)